<a href="https://colab.research.google.com/github/visheshagrawal/Deep_Learning_Sequences/blob/master/ffn_multi_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import io
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets,transforms
import torch.optim as optim
from torch.autograd import Variable

tr=datasets.MNIST("",train=True,download=True,transform=transforms.Compose([transforms.ToTensor()]))
train = np.load('reduced_train.npz')
x_train = train['X_train']
# x_train = torch.from_numpy(x_train)
y_train = train['y_train']
# y_train = torch.from_numpy(y_train)
mask_train = train['mask_train']
#print(x_train)


In [ ]:
z_train = torch.utils.data.TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
print(z_train[0])

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor(6.))


In [ ]:
val = np.load('reduced_val.npz')
x_val = val['X_val']
#x_val = torch.from_numpy(x_val)
y_val = val['y_val']
#y_val = torch.from_numpy(y_val)
mask_val = val['mask_val']
print(x_val.shape)

(635, 400, 20)


In [ ]:
z_test = torch.utils.data.TensorDataset(torch.Tensor(x_val), torch.Tensor(y_val))
print(z_test[0])

(tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.2400, 0.0000, 0.2700,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]), tensor(6.))


In [ ]:
train_loader= torch.utils.data.DataLoader(z_train,64,True)
#print(z_train)
#print(train_loader)
test_loader= torch.utils.data.DataLoader(z_test,64,False)

In [ ]:
batch_size = 64
seq_len = 400
n_feat = 20
n_hid = 30
n_class = 10
lr = 0.0025
n_filt = 10
drop_prob = 0.5

In [ ]:
for batch_idx,(data,target) in enumerate(train_loader):
  print(batch_idx)
  print(data.view(64,-1).size())
  print(target.size())
  break


0
torch.Size([64, 8000])
torch.Size([64])


In [ ]:
for data,target in test_loader:
  print(data.view(64,-1).size())
  print(target.size())
  break

torch.Size([64, 8000])
torch.Size([64])


In [ ]:
model=nn.Sequential(nn.Linear(8000,n_hid),nn.ReLU(),nn.Linear(n_hid,n_class),nn.LogSoftmax(dim=1))

criterion= nn.NLLLoss()

optimizer=optim.Adam(model.parameters(),lr=0.003)

epochs=10

for epoch in range(epochs):
  running_loss=0
  for batch_idx,(data,target) in enumerate(train_loader):
    data=data.view(-1,8000) 
    target=target.type(torch.LongTensor)
    # print(data.shape)
    # print(target)
    data, target = Variable(data), Variable(target)
    # print(data)
    # print(target)
    optimizer.zero_grad()
    output = model(data)
    loss=criterion(output,target)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if batch_idx % 10 == 0:
    #if True:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
      epoch, batch_idx * len(data), len(train_loader.dataset),
      100. * batch_idx / len(train_loader), loss.item()))
  
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data=data.view(-1,8000)
    target=target.type(torch.LongTensor)
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    # sum up batch loss
    test_loss += criterion(output,target)
    # get the index of the max log-probability
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))
  print("Epoch finished")


Train Epoch: 0 [0/2423 (0%)]	Loss: 2.339217
Train Epoch: 0 [640/2423 (26%)]	Loss: 1.889902
Train Epoch: 0 [1280/2423 (53%)]	Loss: 1.645668
Train Epoch: 0 [1920/2423 (79%)]	Loss: 1.571895


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



Test set: Average loss: 0.0232, Accuracy: 307/635 (48%)

Epoch finished
Train Epoch: 1 [0/2423 (0%)]	Loss: 1.204122
Train Epoch: 1 [640/2423 (26%)]	Loss: 0.970637
Train Epoch: 1 [1280/2423 (53%)]	Loss: 0.923423
Train Epoch: 1 [1920/2423 (79%)]	Loss: 0.710859

Test set: Average loss: 0.0163, Accuracy: 448/635 (71%)

Epoch finished
Train Epoch: 2 [0/2423 (0%)]	Loss: 0.562313
Train Epoch: 2 [640/2423 (26%)]	Loss: 0.542977
Train Epoch: 2 [1280/2423 (53%)]	Loss: 0.347559
Train Epoch: 2 [1920/2423 (79%)]	Loss: 0.396190

Test set: Average loss: 0.0136, Accuracy: 468/635 (74%)

Epoch finished
Train Epoch: 3 [0/2423 (0%)]	Loss: 0.213985
Train Epoch: 3 [640/2423 (26%)]	Loss: 0.241018
Train Epoch: 3 [1280/2423 (53%)]	Loss: 0.187848
Train Epoch: 3 [1920/2423 (79%)]	Loss: 0.154809

Test set: Average loss: 0.0125, Accuracy: 475/635 (75%)

Epoch finished
Train Epoch: 4 [0/2423 (0%)]	Loss: 0.108677
Train Epoch: 4 [640/2423 (26%)]	Loss: 0.107275
Train Epoch: 4 [1280/2423 (53%)]	Loss: 0.082676
Train Ep

In [ ]:
  for epoch in range(epochs):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
      data=data.view(-1,8000)
      target=target.type(torch.LongTensor)
      data, target = Variable(data, volatile=True), Variable(target)
      output = model(data)
      # sum up batch loss
      test_loss += criterion(output,target)
      # get the index of the max log-probability
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  



Test set: Average loss: 0.0130, Accuracy: 480/635 (76%)


Test set: Average loss: 0.0130, Accuracy: 480/635 (76%)


Test set: Average loss: 0.0130, Accuracy: 480/635 (76%)


Test set: Average loss: 0.0130, Accuracy: 480/635 (76%)


Test set: Average loss: 0.0130, Accuracy: 480/635 (76%)


Test set: Average loss: 0.0130, Accuracy: 480/635 (76%)


Test set: Average loss: 0.0130, Accuracy: 480/635 (76%)


Test set: Average loss: 0.0130, Accuracy: 480/635 (76%)


Test set: Average loss: 0.0130, Accuracy: 480/635 (76%)


Test set: Average loss: 0.0130, Accuracy: 480/635 (76%)

